## [GLOBAL IMPORTS]

In [1]:
# m1 error fix
import sentencepiece

from fastai.data.external import (
    untar_data, 
    URLs,
)

from fastai.data.core import L

from fastai.data.block import (
    DataBlock,
)

from fastai.data.transforms import (
    RandomSplitter, 
    parent_label,
    GrandparentSplitter,
)


from fastai.text.all import (
    defaults,
    # file handler
    get_text_files, parent_label,
    # tabular util
    Tokenizer, WordTokenizer, SubwordTokenizer, Numericalize,
    LMDataLoader, 
    # data block
    TextBlock, CategoryBlock,
    # model
    AWD_LSTM,
    # metric
    Perplexity, accuracy,
    # learner
    language_model_learner,
    text_classifier_learner,
    # debug log
    first, coll_repr
)

from fastai.text.core import replace_rep

from functools import partial

# suppress Cuda device warnings so we don't BLOW UP logs
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

#### pip install

In [55]:
%pip install fastbook

python(19687) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 11.2 MB/s eta 0:00:00a 0:00:01
INFO: pip is looking at multiple versions of tokenizers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 42.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.5/386.5 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.4/295.4 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

#### Inspect source code with `??`

In [2]:

# inspect source
# Tokenizer??
replace_rep??

Signature: replace_rep(t)
Source:   
def replace_rep(t):
    "Replace repetitions at the character level: cccc -- TK_REP 4 c"
    def _replace_rep(m):
        c,cc = m.groups()
        return f' {TK_REP} {len(cc)+1} {c} '
    return _re_rep.sub(_replace_rep, t)
File:      ~/miniconda3/envs/m1_torch_gpu/lib/python3.10/site-packages/fastai/text/core.py
Type:      function

## NLP Deep Dive 

### RNNs
- Self-supervised learning is training a model using labels that are :
  - **EMBEDDED** in the `independent` variable
    - training to predict the next word in a text is an example
  - rather than requiring EXTERNAL labels


### ULMFit 
- Universal Language Model Fine-tuning
  - 1 - fine-tune the **sequence based** language model
  - 2 - then fine-tune the **classification** model
- This tends to yield BETTER results

## Text Processing

Predicting sentence length isn't obvious ... correlated to human breathe?
- Sentences can be of different LENGTHS
- Documents can be LONG

Review our background with how a single categorical variable can be used as an
indpendent variable, here's the approach we took for a single categorical var :
- 1 - Make a list of all the possible levels of that categorical var -- vocab --
- 2 - Replace each level with it's `index` in the -- vocab --
- 3 - Create an `embedding matrix` for this containing a row for each item
      .i.e for each item in the -- vocab --
- 4 - Use this `embedding matrix` as the first layer of a neural network  

```sh
        A dedicated **embedding matrix** can take as inputs the raw -- vocab --  
        indexes created in step 2;  
            - this is equivalent to  
            - but FASTER and more EFFICIENT than a matrix that takes as input  
            one-hot-encoded vectors representing the indexes  
```

We can do the same thing ^ with TEXT!  What is new is the idea of a sequence

- 1 - [ Tokenization ]
    - convert text into a list of (depending on granularity) :  
      - characters  
      - substrings - (GPT, HuggingFace)  
      - words  
- 2 - [ Numericalization ]  
    - -- vocab -- list hashed to an index number lookup
- 3 - Language Model [ Data Loader Creation ]   
    - `LMDataLoader` handles creating  
      - `dependent` variable that is  
      - `offset` from the `independent` by ONE `token`  
    - Also handles details such as :  
      - shuffling the training data so that the independent and dependent  
        variable maintain their structure as required  
      - latent breathe?  
- 4 - [ Language Model Creation ]  
    - RNN  
      - handles INPUT lists that can be of ARBITRARY LENGTH  


### -- Tokenization --

Resolution

- 1 - Character
    - split into INDIVDUAL CHARS
- 2 - Subword
    - split into SMALLER parts
    - based on most COMMONLY occuring substrings
        - "occasion" => "o" "c" "ca" "sion"
- 3 - Word
    - apply language specific separator like 'white' space
    - generally punctuation marks are SEPARATE tokens
        - as opposed to totally NEW words

In [3]:
# Get text data from IMDB
# folder structure == path + :
# - train/
# - test/
# - unsup/
path = untar_data(URLs.IMDB)

files = get_text_files(path, folders=['train', 'test', 'unsup'])
txt = files[0].open().read()

files[:3], txt[:175]

((#3) [Path('/Users/mton/.fastai/data/imdb/test/neg/1821_4.txt'),Path('/Users/mton/.fastai/data/imdb/test/neg/9487_1.txt'),Path('/Users/mton/.fastai/data/imdb/test/neg/4604_4.txt')],
 "Alan Rickman & Emma Thompson give good performances with southern/New Orleans accents in this detective flick. It's worth seeing for their scenes- and Rickman's scene with Hal")

In [4]:
# Xform text to tokens
# WordTokenizer is a Fastai tokenizing library that can : 
# - collect subwords
# - handling contextual corner cases like '.' as sentence end vs value marker
spacy = WordTokenizer()
toks = first(spacy([txt]))

# @audit : explain coll_repr
print(coll_repr(toks, 30))

# The char '.' is terminated in a sentence, but not the '1.00' acronym
# Tokenization logic needs to handle very subtle context
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#121) ['Alan','Rickman','&','Emma','Thompson','give','good','performances','with','southern','/','New','Orleans','accents','in','this','detective','flick','.','It',"'s",'worth','seeing','for','their','scenes-','and','Rickman',"'s",'scene'...]


(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

#### -- Common -- [ Prefixes ( xx ) ]

xx - not common prefix, these are special tokens
- xxbos : Beginning of stream
    - this token indicates the model will learn it needs to "forget" what was  
    said previously and focus on upcoming words
- xxmaj : Indicates the next word begins with a capital 
  (we lower cased everything)
- xxunk : Indicates a word is unknown
- xxrep : !!!!! => `repeated char token` + `!` so we can count repeats as  
  opposed to treating them as unique
- xxwrep : for repeated words as opposed to characters

-- [x] -- Tokenizer(spacy)

@audit : Explain this

In [17]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt)), 31)

(#139) ['xxbos','xxmaj','alan','xxmaj','rickman','&','xxmaj','emma','xxmaj','thompson'...] 31


#### -- FastAi -- [ Text Processing Rules ]

In [18]:
# inspect source code
replace_rep??

# check default rules
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

Signature: replace_rep(t)
Source:   
def replace_rep(t):
    "Replace repetitions at the character level: cccc -- TK_REP 4 c"
    def _replace_rep(m):
        c,cc = m.groups()
        return f' {TK_REP} {len(cc)+1} {c} '
    return _re_rep.sub(_replace_rep, t)
File:      ~/miniconda3/envs/m1_torch_gpu/lib/python3.10/site-packages/fastai/text/core.py
Type:      function

#### -- Subword -- [Vocabulary Size]

Subword tokenization provides an easy way to :
- easily scale between character and word tokenization
- handles EVERY human language (not just white space separated)
    - including music and genomic sequences

Vocabulary Size is a trade-off between :

- Larger - fewer tokens per sentences
    - faster training
    - less state
    - downside : LARGER EMBEDDING MATRIX
        - requires MORE data to LEARN

In [19]:
txts = L(o.open().read() for o in files[:2])
txts

(#2) ["Alan Rickman & Emma Thompson give good performances with southern/New Orleans accents in this detective flick. It's worth seeing for their scenes- and Rickman's scene with Hal Holbrook. These three actors mannage to entertain us no matter what the movie, it seems. The plot for the movie shows potential, but one gets the impression in watching the film that it was not pulled off as well as it could have been. The fact that it is cluttered by a rather uninteresting subplot and mostly uninteresting kidnappers really muddles things. The movie is worth a view- if for nothing more than entertaining performances by Rickman, Thompson, and Holbrook.",'I have seen this movie and I did not care for this movie anyhow. I would not think about going to Paris because I do not like this country and its national capital. I do not like to learn french anyhow because I do not understand their language. Why would I go to France when I rather go to Germany or the United Kingdom? Germany and the Unit

In [20]:
# @audit : Explain what is going on here
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [10]:
%pip install --user 'sentencepiece!=0.1.90,!=0.1.91'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [13]:
subword(1000)

'▁A lan ▁Rickman ▁ & ▁ E m ma ▁Thompson ▁g i v e ▁go o d ▁ per for m ance s ▁wi th ▁ s ou ther n / N ew ▁O r lea n s ▁ac c'

In [14]:
subword(200)

'▁A lan ▁Rickman ▁ & ▁ E m ma ▁Thompson ▁g i v e ▁go o d ▁ per for m ance s ▁wi th ▁ s ou ther n / N ew ▁O r lea n s ▁ac c'

In [15]:
subword(10_000)

'▁A lan ▁Rickman ▁ & ▁ E m ma ▁Thompson ▁g i v e ▁go o d ▁ per for m ance s ▁wi th ▁ s ou ther n / N ew ▁O r lea n s ▁ac c'

##### Numericalization with Fastai

Numericalization is the process of mapping tokens to integers
- 1 - vocabulary : list of all possible levels of categorical variable
    - RGB captures visual pixel values!
- 2 - replace each level with it's index in the vocab
    - each R, G, B channel has value between 0 - 255

In [30]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#139) ['xxbos','xxmaj','alan','xxmaj','rickman','&','xxmaj','emma','xxmaj','thompson','give','good','performances','with','southern','/','xxmaj','new','xxmaj','orleans','accents','in','this','detective','flick','.','xxmaj','it',"'s",'worth','seeing'...]


In [31]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#139) ['xxbos','xxmaj','alan','xxmaj','rickman','&','xxmaj','emma','xxmaj','thompson'...]

In [32]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab, 20)


"(#32) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','.','the','i','and','not','to','it','movie','this','for',','...]"

In [33]:
nums = num(toks)[:20]
nums

TensorText([ 2,  8,  0,  8, 22,  0,  8,  0,  8,  0,  0,  0,  0,  0,  0,  0,  8,
             0,  8,  0])

In [34]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj xxunk xxmaj rickman xxunk xxmaj xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxmaj xxunk'

##### Putting Our Texts into Batches for a Language Model

With IMAGEs for batching we needed to :

- RESIZE height and width
- so we could group and stack them in a single tensor

With TEXT
- can't resize arbitrarily varied length to fix length
- char order matters to predict next token
- each NEW batch MUST begin precisely where the old batch finished



In [39]:
nums200 = toks200.map(num)

dl = LMDataLoader(nums200)

x,y = first(dl)
x.shape, y.shape

(torch.Size([64, 4]), torch.Size([64, 4]))

In [40]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj xxunk xxmaj'

In [41]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj xxunk xxmaj rickman'

## -- Training a Text Classifier --

- 1 - fine-tune our `language model` trained on Wikipedia
- 2 - use that model to train our `classifier`

### Language Model Using DataBlock

In [42]:
get_imdb = partial(
    get_text_files,
    folders = ['train', 'test', 'unsup']
)

In [43]:
# FileNotFoundError: [Errno 2] No such file or directory: 
# '.fastai/data/imdb_tok/counter.pkl'
# This can occur if we cancel this tok process : 
# - it'll cache in a malformed
# - when we rerun, it only checks that the 'imdb_tok' folder exists
# - and tries to load the `counter.pkl` that never actually got completed
dls_lm = DataBlock(
    # @audit : Explain is_lm
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, 
    splitter=RandomSplitter(0.1)
# ).dataloaders(path, path=path, bs=128, seq_len=80)
# stepping down to fit on M1 mac
).dataloaders(path, path=path, bs=32, seq_len=32)

In [44]:
dls_lm.show_batch(max_n=2)

,text,text_
0,xxbos xxmaj this movies shook my will to live why this abomination is n't the bottom 100 list i do n't know . \n\n xxmaj my life was saved by the healing,xxmaj this movies shook my will to live why this abomination is n't the bottom 100 list i do n't know . \n\n xxmaj my life was saved by the healing power
1,fact that xxup rotor can take off his sunglasses xxup and xxup see xxup into xxup the xxup past ! xxmaj apparently a function called xxup sensor xxup recall was built into,that xxup rotor can take off his sunglasses xxup and xxup see xxup into xxup the xxup past ! xxmaj apparently a function called xxup sensor xxup recall was built into his


### Fine-Tuning the Language Model



In [45]:
learn = language_model_learner(
    dls_lm,
    AWD_LSTM,
    drop_mult=0.3,
    metrics=[accuracy, Perplexity()]
).to_fp16()

#### [x] Load Epoch else ~ 1 day of GPU poor training

In [46]:
# @note : takes 940 minutes to train
# epoch	train_loss	valid_loss	accuracy	perplexity	time
#     0	4.004853	3.903127	0.300107	49.557171	15:40:19

# @note : takes 351 minutes to train
# epoch	train_loss	valid_loss	accuracy	perplexity	time
#     0	4.211511	4.084476	0.288052	59.410797	5:51:50
# learn.fit_one_cycle(1, 2e-2)

# Path('/Users/mton/.fastai/data/imdb/models/1epoch.pth')
# learn.save('1epoch')

# Let's load our model instead of performing that MASSIVE training
learn = learn.load('1epoch')

In [27]:
# Finetune after initial model training done
# @audit-ok : 3931 minutes of training lol

# learn.unfreeze()
# learn.fit_one_cycle(10, 2e-3)

/Users/mton/miniconda3/envs/m1_torch_gpu/lib/python3.10/site-packages/torch/amp/autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/Users/mton/miniconda3/envs/m1_torch_gpu/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py:120: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.914853,3.895737,0.308965,49.192291,6:45:36
1,3.990270,3.962281,0.304625,52.577129,4:51:24
2,3.994842,3.950320,0.306160,51.952000,4:32:56
3,3.931293,3.920458,0.309763,50.423538,5:51:19
4,3.829621,3.875810,0.314348,48.221737,4:42:34
5,3.826797,3.819576,0.319951,45.584888,20:24:10
6,3.744042,3.756420,0.326571,42.794952,5:05:19
7,3.695207,3.704155,0.332006,40.615730,4:38:18
8,3.607924,3.672139,0.336082,39.335972,4:18:45
9,3.586068,3.668146,0.336638,39.179214,4:21:06


#### [x] Load Encoder else ~ 3 days of Mac compute lol

In [47]:
# learn.save_encoder('finetuned')
learn.load_encoder('finetuned')

### Text Generation

In [48]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2

preds = [
    learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)
]

print("\n".join(preds))

i liked this movie because the credits rolled de delegate ball cos it was intelligent enough to keep me entertained throughout atleast portions catfights hairy abnormally mythology . i mean d rd pottawatomie reminded me linked fella \ glove there`s sorted hinds looses pajamas woolvett
i liked this movie because i saw it yesterday 2006 grisham attended by widowed parents skewered 55 bucks by catherine phantasmagorical stocked hoard milk masters grasse proletariat mid-'80s lace schellenberg subs 0.0 currents . i was told japanese askwith tee raincoat highlighting brassed herding 1600


### Fine-Tuning the Classifier Model

We're now moving :
- from Language Model Finetuning
- to Classifier Finetuning

The Fundamental Difference is :
- Classifier predicts an EXTERNAL LABEL
- Language Model predicts next token (char/word)

In [49]:
dls_clas = DataBlock(
    blocks=(
        # is_lm isn't explicitly set here, so DEFAULTS to FALSE
        # effectively configures TextBlock to use regular labeled data as 
        # opposed to next tokens as labels
        # @audit : Explain like I am 5 ^
        # Sorting and padding is AUTOMATICALLY done by Fast AI data block API
        # when `is_lm` = FALSE
        TextBlock.from_folder(
            path,
            # vocab created for language model fine tuning is passed in here
            # BECAUSE we want to make sure to use the same correspondence to
            # token
            # @audit : Explain like I am 5 ^
            vocab=dls_lm.vocab,
        ), 
        CategoryBlock
    ),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
# ).dataloaders(path, path=path, bs=128, seq_len=72)
).dataloaders(path, path=path, bs=64, seq_len=64) # addressing out of memory

#### [x] MPS memory limit issue

```sh
RuntimeError: MPS backend out of memory (MPS allocated: 2.16 GB, other  
allocations: 6.95 GB, max allowed: 9.07 GB). Tried to allocate 2.44 MB on  
private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit  
for memory allocations (may cause system failure).
```

Going to :
- reduce bs=128 > bs=64
- reduce seq_len=72 > seq_len=64

##### ChatGPT check

> Yes, reducing both the batch size (bs) and the sequence length (seq_len) can  
help resolve out-of-memory (OOM) errors when training neural networks, including  
those you are working with in FastAI. Here's how these changes can alleviate  
memory issues:

> Reducing Batch Size:
- From bs=128 to bs=64: The batch size determines how many examples you process  
at once. Reducing the batch size decreases the amount of GPU memory required  
during training because fewer data points means less information needs to be  
stored in memory at any one time.

> Reducing Sequence Length:
- From seq_len=72 to seq_len=64: The sequence length in natural language  
processing tasks dictates the length of the sequences that your model processes.  
By shortening the maximum sequence length, each input tensor contains fewer  
tokens, which in turn reduces the memory footprint for both the model inputs and  
the intermediate activations within the model.

> By making these changes, you're instructing FastAI to load less data into  
memory for each training step, which should help you stay within the memory  
limits of your MPS backend.

> However, keep in mind that reducing the batch size might impact the gradient  
estimates during training, potentially affecting the convergence and stability  
of the training process. Similarly, reducing the sequence length might impact  
the context the model can see, which could affect performance if the model  
relies on longer sequences to make predictions. It's often a balance between  
resource constraints and model performance.

> If you apply these changes and still encounter memory issues, you might need  
to consider further reducing the batch size or sequence length, optimizing your  
model architecture, or using a machine with more memory resources.

> Remember to watch out for any changes in model performance as you make these  
adjustments, and validate that the model still learns effectively with the new  
settings.

In [50]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos xxmaj by now you 've probably heard a bit about the new xxmaj disney dub of xxmaj miyazaki 's classic film , xxmaj laputa : xxmaj castle xxmaj in xxmaj the xxmaj sky . xxmaj during late summer of 1998 , xxmaj disney released "" kiki 's xxmaj delivery xxmaj service "" on video which included a preview of the xxmaj laputa dub saying it was due out in "" 1 xxrep 3 9 "" . xxmaj it 's obviously way past that year now , but the dub has been finally completed . xxmaj and it 's not "" laputa : xxmaj castle xxmaj in xxmaj the xxmaj sky "" , just "" castle xxmaj in xxmaj the xxmaj sky "" for the dub , since xxmaj laputa is not such a nice word in xxmaj spanish ( even though they use the word xxmaj laputa many times",pos
2,"xxbos xxmaj titanic directed by xxmaj james xxmaj cameron presents a fictional love story on the historical setting of the xxmaj titanic . xxmaj the plot is simple , xxunk , or not for those who love plots that twist and turn and keep you in suspense . xxmaj the end of the movie can be figured out within minutes of the start of the film , but the love story is an interesting one , however . xxmaj kate xxmaj winslett is wonderful as xxmaj rose , an aristocratic young lady betrothed by xxmaj cal ( billy xxmaj zane ) . xxmaj early on the voyage xxmaj rose meets xxmaj jack ( leonardo dicaprio ) , a lower class artist on his way to xxmaj america after winning his ticket aboard xxmaj titanic in a poker game . xxmaj if he wants something , he goes and gets it",pos


In [51]:
nums_samp = toks200[:10].map(num)

nums_samp.map(len)

(#2) [139,152]

In [52]:
learn = text_classifier_learner(
    dls_clas,
    AWD_LSTM,
    drop_mult=0.5,
    metrics=accuracy,
).to_fp16()

### FineTuning the Classifier 

In [79]:
learn = learn.load_encoder('finetuned')

In [80]:
learn.fit_one_cycle(1, 2e-2)

# epoch	train_loss	valid_loss	accuracy	time
#     0	0.320355	0.216716	0.914800	4:48:23
# @ 288 minutes

epoch,train_loss,valid_loss,accuracy,time


loc("outputTensor"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/495c257e-668e-11ee-93ce-926038f30c31/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":745:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x64x1x1401xi1>'
loc("outputTensor"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/495c257e-668e-11ee-93ce-926038f30c31/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":745:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x64x1x406xi1>'
loc("outputTensor"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/495c257e-668e-11ee-93ce-926038f30c31/Library/Caches/com.apple.xbs/Sources/MetalPe

In [73]:
learn.freeze_to(-2)

In [74]:
learn.fit_one_cycle(1, slice(1e-2/(2.6**4)), 1e-2)

epoch,train_loss,valid_loss,accuracy,time


RuntimeError: MPS backend out of memory (MPS allocated: 2.16 GB, other allocations: 6.95 GB, max allowed: 9.07 GB). Tried to allocate 2.44 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).